In [43]:
import re
import nltk
import pandas as pd
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
file = 'receitinhas.csv'
df = pd.read_csv(file, index_col=0)

print(df)

In [ ]:
df.dtypes

<h2>Limpeza</h2>

In [40]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maisamoreira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maisamoreira/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/maisamoreira/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maisamoreira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def clean_ingredients(ingredients_text):
    ingredients_text = re.sub(r"[\{\}]", "", ingredients_text).strip()
    ingredients_list = [item.strip() for item in ingredients_text.split(',')]
    return ingredients_list

In [ ]:
df['Ingredientes'] = df['Ingredientes'].apply(clean_ingredients)
print(df)

Removendo números

In [ ]:
def remove_numbers(text):
    return re.sub(r'\d', '', str(text))

def clean_numbers(text):
    try:
        ingredients_list = eval(text)
        if isinstance(ingredients_list, list):
            return [remove_numbers(item) for item in ingredients_list]
    except:
        return remove_numbers(text)
    return text

df['Ingredientes'] = df['Ingredientes'].apply(clean_numbers)

In [ ]:
print(df)

Todas as letras em minúsculas

In [14]:
def convert_lowercase(text):
    try:
        ingredients_list = eval(text)
        if isinstance(ingredients_list, list):
            return [str(item).lower() for item in ingredients_list]
    except:
        return str(text).lower()
    return text

df['Ingredientes'] = df['Ingredientes'].apply(convert_lowercase)

In [15]:
print(df)

                                     Receita  \
0                             Massa de Pizza   
1                 Receita de Bolo de Cenoura   
2               Bolo de Chocolate Molhadinho   
3                Como Fazer Bolinho de Chuva   
4                               Bolo de Fubá   
5                              Bolo de Milho   
6                         Fricassê de Frango   
7                        Bolo de Aniversário   
8                             Bolo de Banana   
9                                  Panquecas   
10                 Pudim de Leite Condensado   
11                   Torta de Liquidificador   
12                           Cuscuz Paulista   
13                       Pãozinho de Cenoura   
14  Pão Integral Caseiro com Aveia e Linhaça   
15                    Pão Estrela de Nutella   
16                  Bolo Prestígio Econômico   
17                            Muffin de Maçã   
18     Bolo de Cenoura Fofinho com Cobertura   
19             Torta de Rúcula e Tomate 

In [ ]:
# def identify_nouns(ingredient):
#     if pd.isna(ingredient):
#         return ingredient
#     doc = nlp(str(ingredient))
#     nouns = [token.text for token in doc if token.pos_ in ('NOUN', 'PROPN')]
#     return ' '.join(nouns)

# df_final = apply_in_all_dataset(df_final, identify_nouns)

Remover palavras de medidas e stopwords

In [35]:

stop_words = set(stopwords.words('portuguese'))

def clean_measurements_and_stopwords(ingredients):
    cleaned_ingredients = []
    for item in ingredients:
        # incluir café?
        item = re.sub(r"\d+|\([^)]*\)|\\|/|\b(bem|bola|bolas|caixa|caixas|caixinha|caixinhas|chá|colher|colheres|copo|copos|cozida|cozidas|cozido|cozidos|crua|cruas|decorar|dente|dentes|desfiado|desfiados|envelope|envelopes|fatia|fatias|forte|gosto|grosso|grossa|\bgr\b|grs|grama|gramas|kg|kgs|lactose|lata|latas|litro|litros|maço|maços|magro|média|médias|ml|mls|mililitro|mililitros|morno|orgânico|orgânicos|pacote|pacotes|peneirada|peneiradas|picado|picados|picada|picadas|picadinha|picadinhas|pincelar|pitada|pitadas|polvilhar|quilo|quilos|raspa|raspas|refogar|rodela|rodelas|sobremesa|sopa|sopas|tablete|tabletes|unidade|unidades|xícara|xícaras|zero)\b", "", item)
        item = item.replace('-', ' ')
        
        tokens = [word for word in item.split() if word not in stop_words]
        cleaned_ingredients.append(" ".join(tokens))
    
    return cleaned_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(clean_measurements_and_stopwords)

In [46]:
print(df)

                                     Receita  \
0                             Massa de Pizza   
1                 Receita de Bolo de Cenoura   
2               Bolo de Chocolate Molhadinho   
3                Como Fazer Bolinho de Chuva   
4                               Bolo de Fubá   
5                              Bolo de Milho   
6                         Fricassê de Frango   
7                        Bolo de Aniversário   
8                             Bolo de Banana   
9                                  Panquecas   
10                 Pudim de Leite Condensado   
11                   Torta de Liquidificador   
12                           Cuscuz Paulista   
13                       Pãozinho de Cenoura   
14  Pão Integral Caseiro com Aveia e Linhaça   
15                    Pão Estrela de Nutella   
16                  Bolo Prestígio Econômico   
17                            Muffin de Maçã   
18     Bolo de Cenoura Fofinho com Cobertura   
19             Torta de Rúcula e Tomate 

In [47]:
df.to_csv('ingredientes_parciais.csv')

Lematização

In [48]:
lemmatizer = WordNetLemmatizer()

def lemmatize_word(word):
    return lemmatizer.lemmatize(word, wordnet.NOUN)

def lemmatize_ingredients(ingredients):
    lemmatized_ingredients = []
    for item in ingredients:
        tokens = word_tokenize(item)
        lemmatized_tokens = [lemmatize_word(word.lower()) for word in tokens]
        lemmatized_ingredients.append(" ".join(lemmatized_tokens))
    return lemmatized_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(lemmatize_ingredients)

print(df)


                                     Receita  \
0                             Massa de Pizza   
1                 Receita de Bolo de Cenoura   
2               Bolo de Chocolate Molhadinho   
3                Como Fazer Bolinho de Chuva   
4                               Bolo de Fubá   
5                              Bolo de Milho   
6                         Fricassê de Frango   
7                        Bolo de Aniversário   
8                             Bolo de Banana   
9                                  Panquecas   
10                 Pudim de Leite Condensado   
11                   Torta de Liquidificador   
12                           Cuscuz Paulista   
13                       Pãozinho de Cenoura   
14  Pão Integral Caseiro com Aveia e Linhaça   
15                    Pão Estrela de Nutella   
16                  Bolo Prestígio Econômico   
17                            Muffin de Maçã   
18     Bolo de Cenoura Fofinho com Cobertura   
19             Torta de Rúcula e Tomate 

In [49]:
df.to_csv('ingredientes_parciais2.csv')

<h2>Análise</h2>

In [ ]:
def collect_words(df, columns):
    all_words = []
    for column in columns:
        for text in df[column].dropna():
            words= re.findall(r'\b\w+\b', text.lower())
            all_words.extend(words)
    return all_words

columns_ingredients = [col for col in df_final.columns if col.startswith('Ingredientes')]
all_words_dataset = collect_words(df_final, columns_ingredients)

count_words = Counter(all_words_dataset)

print(count_words)  